In [1]:
import pandas as pd

import json
import glob
import os

In [2]:
base = "/amd.home/home/i11/zeitz/experiments/chpot/"
paths = glob.glob(base + "building_blocks/*.json")
data = [json.load(open(path)) for path in paths]

In [7]:
queries = pd.DataFrame.from_records([{ 
    'features': run['feature_flags'].replace('DEFAULT, TDCCH_APPROX, TDCCH_POSTCUSTOMIZATION, TDCCH_PRECUSTOMIZATION, TDCCH_QUERY_ASTAR, TDCCH_TRIANGLE_SORTING', ''), 
    **algo } 
    for run in data for exp in run.get('experiments', []) for algo in exp['algo_runs'] if algo.get('algo') == 'CH Potentials Query'])

In [8]:
queries.columns

Index(['features', 'algo', 'from', 'lower_bound', 'num_pot_evals',
       'num_queue_pops', 'num_relaxed_arcs', 'result', 'running_time_ms',
       'to'],
      dtype='object')

In [12]:
table = queries.groupby('features')[['running_time_ms', 'num_queue_pops', 'num_relaxed_arcs']].mean()
table['num_relaxed_arcs'] = table['num_relaxed_arcs'] / 1000000
table['num_queue_pops'] = table['num_queue_pops'] / 1000
print(table.to_latex())
table

\begin{tabular}{lrrr}
\toprule
{} &  running\_time\_ms &  num\_queue\_pops &  num\_relaxed\_arcs \\
features                                           &                  &                 &                   \\
\midrule
                                                   &       315.702213 &      445.028300 &          2.227082 \\
CHPOT\_NO\_DEG2, CHPOT\_NO\_DEG3,                      &       469.180056 &      905.298525 &          2.505806 \\
CHPOT\_NO\_DEG2, CHPOT\_NO\_DEG3, CHPOT\_NO\_SCC,        &       547.108964 &     1006.468535 &          2.777881 \\
CHPOT\_NO\_DEG2, CHPOT\_NO\_DEG3, CHPOT\_NO\_SCC, CHP... &      1765.856428 &     7087.019915 &         19.607343 \\
CHPOT\_NO\_DEG2, CHPOT\_NO\_DEG3, CHPOT\_ONLY\_TOPO,     &      1308.088898 &     6249.891000 &         17.364401 \\
CHPOT\_NO\_DEG3,                                     &       410.222530 &      767.564570 &          2.406106 \\
CHPOT\_NO\_DEG3, CHPOT\_ONLY\_TOPO,                    &      1180.829687 &     5233.74318

,running_time_ms,num_queue_pops,num_relaxed_arcs
features,,,
,315.702213,445.028300,2.227082
"CHPOT_NO_DEG2, CHPOT_NO_DEG3,",469.180056,905.298525,2.505806
"CHPOT_NO_DEG2, CHPOT_NO_DEG3, CHPOT_NO_SCC,",547.108964,1006.468535,2.777881
"CHPOT_NO_DEG2, CHPOT_NO_DEG3, CHPOT_NO_SCC, CHPOT_ONLY_TOPO,",1765.856428,7087.019915,19.607343
"CHPOT_NO_DEG2, CHPOT_NO_DEG3, CHPOT_ONLY_TOPO,",1308.088898,6249.891000,17.364401
"CHPOT_NO_DEG3,",410.222530,767.564570,2.406106
"CHPOT_NO_DEG3, CHPOT_ONLY_TOPO,",1180.829687,5233.743180,16.557924
"CHPOT_ONLY_TOPO,",901.837320,3035.598595,15.142951
